In [ ]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import numpy as np

from keras import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.optimizers import Adam

In [ ]:
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same")) 
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    # compile model
    adam_opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    return model

def load_and_norm_samples():
    (train_data, _), (_, _) = load_data()
    training_images = np.expand_dims(train_data, axis = -1)
    training_images = training_images.astype('float32')
    training_images = training_images / 255.0
    return training_images
    
def generate_real_samples(dataset, num_samples):
    idx = np.random.randint(0, dataset.shape[0], num_samples)
    gen_data = dataset[idx]
    gen_labels = np.ones((num_samples, 1))
    return gen_data, gen_labels
    
def generate_fake_images(num_samples):
    gen_data = np.random.rand(28 * 28 * num_samples)
    gen_data = gen_data.reshape((num_samples, 28, 28, 1))
    gen_labels = np.zeros((num_samples, 1))
    return gen_data, gen_labels

def train_discriminator(model, dataset, num_periods=100, batch_size=256):
    half_batch_size = int(batch_size / 2)
    for epoch in range(num_periods):
        image_real, label_real = generate_real_images(dataset, half_batch_size)
        _, real_acc = model.train_on_batch(image_real, label_real)
        image_fake, label_fake = generate_fake_images(half_batch_size)
        _, fake_acc = model.train_on_batch(image_fake, label_fake)
        print('>%d real=%.0f%% fake=%.0f%%' % (epoch + 1, real_acc * 100, fake_acc * 100))

In [ ]:
def define_generator(latent_dim):
    model = Sequential()
    num_nodes = 128 * 7 * 7
    model.add(Dense(num_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model

def generate_latent_points(latent_dim, num_samples):
    train_input = np.random.randn(latent_dim * num_samples)
    train_input = train_input.reshape(num_samples, latent_dim)
    return train_input

def generate_fake_samples(g_model, latent_dim, num_samples):
    train_input = generate_latent_points(latent_dim, num_samples)
    training_images = g_model.predict(train_input)
    training_labels = np.zeros((num_samples, 1))
    return training_images, training_labels

In [ ]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    # compile model
    adam_opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=adam_opt)
    return model

def train_gan(gan_model, latent_dim, num_epochs=100, n_batch=256):
    for i in range(num_epochs):
        image_gan = generate_latent_points(latent_dim, n_batch)
        labels_gan = ones((n_batch, 1))
        gan_model.train_on_batch(image_gan, labels_gan)

def train(g_model, d_model, gan_model, dataset, latent_dim, num_epochs=100, num_batch=256):
    batch_per_epoch = int(dataset.shape[0] / num_batch)
    half_batch = int(num_batch / 2)
    for i in range(num_epochs):
        for j in range(batch_per_epoch):
            images_real, labels_real = generate_real_samples(dataset, half_batch)
            images_fake, labels_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            images, labels = np.vstack((images_real, images_fake)), np.vstack((labels_real, labels_fake))
            disc_loss, _ = d_model.train_on_batch(images, labels)
            images_gan = generate_latent_points(latent_dim, num_batch)
            labels_gan = np.ones((num_batch, 1))
            gan_loss = gan_model.train_on_batch(images_gan, labels_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, batch_per_epoch, disc_loss, gan_loss))
        if (i+1)%10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)


def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, num_samples=100):
    images_real, labels_real = generate_real_samples(dataset, num_samples)
    _, acc_real = d_model.evaluate(images_real, labels_real, verbose=0)
    images_fake, labels_fake = generate_fake_samples(g_model, latent_dim, num_samples)
    _, acc_fake = d_model.evaluate(images_fake, labels_fake, verbose=0)
    print('Accuracy: real-%.0f% fake-%.0f%' % (acc_real, acc_fake))
    save_plot(images_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

def save_plot(examples, epoch, n=10):
    for i in range(n*n):
        plt.subplot(n, n, i+1)
        plt.axis('off')
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()
  

In [ ]:
latent_dim = 100
discriminator_model = define_discriminator()
generator_model = define_generator(latent_dim)
gan_model = define_gan(generator_model, discriminator_model)
dataset = load_and_norm_samples()
train(generator_model, discriminator_model, gan_model, dataset, latent_dim)